In [2]:
import numpy as np
from numpy import random as rd
from pykrx import stock

#아래 티커는 각각 kodex레버리지 / KODEX 인버스 / kosef미국달러선물 / KODEX 모멘텀PLUS
ETF_ticker = ["122630", "114800", "138230", "244620"]
N = len(ETF_ticker) #ETF 갯수만큼 자산 총 갯수는 N개로 제한된다.
stocks_data = []
srt = "20140101"
end = "20191231"
for i in range(0, N):
    stocks_data.append(stock.get_market_ohlcv_by_date(srt, end, ETF_ticker[i])["종가"][-300:])#룩백윈도우는 300일


In [3]:
#var - cov matrix 생성
def getCov(a, b):
    return np.cov(a,b)

var_cov_mat = np.identity(N) # N * N 단위행렬

for i in range(0, N):
    for j in range(0, N):
        var_cov_mat[i][j] = getCov(stocks_data[i], stocks_data[j])[0,1]

In [9]:
w = [1/N for i in range(0,N)]

In [10]:
risk = np.dot(np.dot(np.transpose(w), var_cov_mat), w) # sigma _ w

In [4]:
import scipy as sp
from scipy import optimize

def eq_constraint(w):
    summation = 0
    for i in w:
        summation += i
    return summation -1

def flarray(w):
    summation = 0
    risk = np.dot(np.dot(np.transpose(w), var_cov_mat), w) # sigma _ w
    N = len(w)
    for wi, i in zip(w, range(0,N)):
        summation += (wi - risk/(N* np.array(np.dot(var_cov_mat, w)[i])))**2
    return summation


In [5]:
w_init = [1/N for i in range(0,N)] # 초기값은 동일비중[1/4 1/4 1/4 1/4]
result1 = sp.optimize.fmin_slsqp(flarray, w_init, eqcons=[eq_constraint])

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 5.900496783761678e-08
            Iterations: 15
            Function evaluations: 98
            Gradient evaluations: 15


In [8]:
#초기값을 달리했을때
w_another_init = [0.1, 0.4, 0.3, 0.2]
result2 = sp.optimize.fmin_slsqp(flarray, w_another_init, eqcons=[eq_constraint])

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 1.4702844042180107e-07
            Iterations: 21
            Function evaluations: 139
            Gradient evaluations: 21


In [9]:
print(result1)
print(result2)
print(result1 - result2)

[-0.2780899   1.04054416  0.77073144 -0.5331857 ]
[-0.27819215  1.04025756  0.7710139  -0.53307931]
[ 0.00010225  0.0002866  -0.00028247 -0.00010638]


In [19]:
w_extreme = [0.4 , 0.1, 0.3, 0.2]
sp.optimize.fmin_slsqp(flarray, w_init, eqcons=[eq_constraint])

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 5.900496783761678e-08
            Iterations: 15
            Function evaluations: 98
            Gradient evaluations: 15


array([-0.2780899 ,  1.04054416,  0.77073144, -0.5331857 ])